<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW2/blob/main/NLP_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/MyDrive/NLP


##Extracting essential data



In [3]:
!pip install -q hazm
!pip install -q dadmatools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.6/862.6 KB 39.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 KB 59.5 MB/s eta 

In [4]:
from __future__ import unicode_literals
import hazm

import json
import pickle
import itertools

from tqdm import tqdm

In [5]:
with open('Nlp HW2/extracted-phrases.json','r',encoding='utf-8') as file:
    data = json.load(file)

##Main code

###Config

In [6]:
from dadmatools.models.normalizer import Normalizer

dadma_normalizer = Normalizer(
    full_cleaning=True,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=True,
    remove_html=True,
    remove_stop_word=True,
    replace_email_with="",
    replace_number_with="",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)

dadma_normalizer.normalize("بیا ای عقل کل با من که بردابرد او بینی")

'عقل بردابرد بینی'

In [7]:
lemmatizer = hazm.Lemmatizer()
stemmer = hazm.Stemmer()

In [11]:
corp = list(set([x['word'] for x in data]))

dual_corp = [phrase for phrase in corp if " " in phrase]
main_corp = [lemmatizer.lemmatize(word) for word in corp if " " not in word]

print(f"number of single words within data is about : {len(main_corp)} and multiple part words count as {len(dual_corp)}")
print("----------------------\n")
print ('%-45s' % 'word', '%-16s' % 'compound')
for i in range(15):
    print ('%-50s' % dual_corp[i], '%-16s' % main_corp[i])

number of single words within data is about : 245048 and multiple part words count as 610169
----------------------

word                                          compound        
تهران امروز فیلم                                   ماده            
پیروزی پوکمون                                      خفایا           
فرودگاه برندیواین                                  سینگر           
گوشت خواری                                         امائم           
تپه تخته قلعه                                      حرنفش           
بارش همرفتی                                        بنیاباج         
پی ریج فلوریدا                                     کایماکلی        
انبوه شدن                                          سترانی          
تله جرد                                            ایاها           
آلن دوونشایر                                       منبهه           
خواهری از سفر آرزوها 2                             سحنون           
علی‌اکبر احمدیان                                   سلبوب           
پادش

In [12]:
import dadmatools.pipeline.language as language

pips = 'lem' 
nlp = language.Pipeline(pips)

print(nlp.analyze_pipes(pretty=True))


============================= Pipeline Overview =============================

#   Component   Assigns       Requires   Scores   Retokenizes
-   ---------   -----------   --------   ------   -----------
0   tokenizer                                     True       
                                                             
1   lemmatize   token.lemma                       False      

✔ No problems found.
{'summary': {'tokenizer': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': True}, 'lemmatize': {'assigns': ['token.lemma'], 'requires': [], 'scores': [], 'retokenizes': False}}, 'problems': {'tokenizer': [], 'lemmatize': []}, 'attrs': {'token.lemma': {'assigns': ['lemmatize'], 'requires': []}}}


In [ ]:
with open("corpus.txt","rb") as f:
  corpo = pickle.load(f)

corpo

In [32]:
corpus = main_corp.copy()
corpus.extend(corpo)
corpus = list(set(corpus))
len(corpus)

250004

In [59]:
corpus = list(set(corpo))

In [60]:
corpus.extend(["ند","یم","ید","ی","تان","ش","ت","شان","مان","م"])

In [61]:
args = {
    'corpus':corpus,
    'pipe':nlp,
    'normalizer':dadma_normalizer,
    'lemmatizer':hazm.Lemmatizer(),
    'stemmer':hazm.Stemmer(),
    'translator':None,
    'word_tokenizer':hazm.WordTokenizer(),
    'sent_tokenizer':hazm.SentenceTokenizer()
}

In [100]:
class foreign_word_detector():

  def __init__(
      self,
      args
      ):
    self.corpus = args['corpus']
    self.lang_pipe = args['pipe']
    self.normalizer = args['normalizer']
    self.lemmatizer = args['lemmatizer']
    self.stemmer = args['stemmer']
    self.translator = args['translator']
    self.word_tokenizer = args['word_tokenizer']
    self.sent_tokenizer = args['sent_tokenizer']

  def detect(self, text):
    sentenced = self.sent_tokenizer.tokenize(text)
    normalized = [self.normalizer.normalize(sent) for sent in sentenced]
    lemmed = [str(self.lang_pipe(sent)) for sent in normalized]
    words_list = [self.word_tokenizer.tokenize(sent) for sent in lemmed]
    words_list = list(itertools.chain(*words_list))
    lemmed_words_list = [lemmatizer.lemmatize(word) for word in words_list]

    output = dict()
    out = []

    print(lemmed_words_list)
    for word in lemmed_words_list:
      if word not in self.corpus:
        out.append(word)
        begin = text.find(word)
        output[word] = [begin,begin+len(word)]
    
    return out, output


In [101]:
def run():
  detector = foreign_word_detector(args)

  text = ""
  print("enter your sample text.\nenter an empty string if you want to end the process")
  while True : 
    text = input()
    if text == "": 
      break
    
    out, output = detector.detect(text)
    print(out)
    print(output)


In [102]:
run()

enter your sample text.
enter an empty string if you want to end the process
امروز تسک سختی داشتیم؛ اما تو توانستی کانتریبیوشن خوبی داشته‌باشی.
['تسک', 'سختی', 'داشت#دار', '؛', 'کانتریبیوشن', 'خوبی', 'داشته\u200cباشی']
['تسک', 'داشت#دار', '؛', 'کانتریبیوشن', 'داشته\u200cباشی']
{'تسک': [6, 9], 'داشت#دار': [-1, 7], '؛': [21, 22], 'کانتریبیوشن': [38, 49], 'داشته\u200cباشی': [55, 65]}


KeyboardInterrupt: ignored

In [96]:
str(nlp(dadma_normalizer.normalize("امروز تسک سختی داشتیم اما تو توانستی کانتریبیوشن مناسبی داشته‌باشی؛ تنکس؛")))

'تسک سختی کانتریبیوشن مناسبی داشته\u200cباشی ؛ تنکس ؛ '

problem is: using stemmer, we get lots of errors; while using lemmatizer, pronouns are not ommited from the word.


one way to deal with it would be to apply stemming right after dealing with the word origin.

another way would be to find a way to work with the tokenizer introduced in the next cell, since it gives us specific features.